In [6]:
import numpy as np
import pandas as pd

%config Completer.use_jedi = False

## Datos
<br>


Los datos empleados en este documento representan el uso a nivel horario del sistema de alquiler de bicicletas en la ciudad de Washington D.C. durante los años 2011 y 2012. Además del número de usuarios, el set de datos contiene información sobre las condiciones meteorológicas y sobre los días festivos. La información de cada columna es:

- instant: record index
- dteday : date
- season : season (1:springer, 2:summer, 3:fall, 4:winter)
- yr : year (0: 2011, 1:2012)
- mnth : month ( 1 to 12)
- hr : hour (0 to 23)
- holiday : weather day is holiday or not (extracted from http://dchr.dc.gov/page/holiday-schedule)
- weekday : day of the week (staring on Sunday)
- workingday : if day is neither weekend nor holiday is 1, otherwise is 0.
- weathersit : 
    - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
    - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
    - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
    - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
- temp : Normalized temperature in Celsius. The values are divided to 41 (max)
- atemp: Normalized feeling temperature in Celsius. The values are divided to 50 (max)
- hum: Normalized humidity. The values are divided to 100 (max)
- windspeed: Normalized wind speed. The values are divided to 67 (max)
- casual: count of casual users
- registered: count of registered users
- cnt: count of total rental bikes including both casual and registered



Los datos están disponibles en [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset).

In [7]:
# Descarga y limpieza inicial de datos
# ==============================================================================
datos = pd.read_csv('bike_sharing_dataset.csv')

In [8]:
# Prepraración del dato
# ==============================================================================
# Se renombran las columnas con nombres más descriptivos
datos = datos.rename(
            columns={
                'dteday': 'date_time',
                'yr': 'year',
                'mnth': 'month',
                'hr': 'hour',
                'weathersit': 'weather',
                'cnt': 'users'
            })

# Se renombran los niveles de la variable meteorológica. Solo hay cuatro valores
# de heavy rain, así que se consideran como rain.
datos['weather'] = datos['weather'].replace({
                                        1: 'clear',
                                        2: 'mist',
                                        3: 'rain',
                                        4: 'rain' 
                                    })

# Se desnormaliza la temperatura, humedad y viento
datos['temp'] = datos['temp'] * 41
datos['atemp'] = datos['atemp'] * 50
datos['hum'] = datos['hum'] * 100
datos['windspeed'] = datos['windspeed'] * 67 

# Se une la fecha y la hora ,y se establece cómo índice
datos['date_time'] = datos.apply(
                        lambda row: f"{row['date_time']} {str(row['hour']).zfill(2)}:00:00",
                        axis = 1
                     )
datos['date_time'] = pd.to_datetime(datos['date_time'], format='%Y-%m-%d %H:%M:%S')
datos = datos.set_index('date_time')
datos = datos.asfreq('H')
datos = datos.sort_index() 


# Se eliminan columnas no utilizadas en este ejemplo. Las variables 'month', 'weekday' y
# 'hour' se recalculan tras la imputación de valores ausentes
                            
datos = datos.drop(columns=['instant', 'season', 'year', 'month', 'hour',
                            'weekday', 'casual', 'registered'])

In [9]:
# Verificar que el índice temporal está completo
# ==============================================================================
(datos.index == pd.date_range(start=datos.index.min(),
                              end=datos.index.max(),
                              freq=datos.index.freq)).all()

True

In [10]:
# Identificar si hay registros incompletos
# ==============================================================================
n_missing = len(datos[datos.isnull().any(axis=1)])
print(f"Número de registros incompletos: {n_missing} ({100 * n_missing / len(datos):.2f}%)")

Número de registros incompletos: 165 (0.94%)


Hay un total de 165 horas para las que no se dispone de información. La serie temporal tiene que estar completa para poder aplicar las estrategias de *forcasting* disponibles en la librería skforecast. En este caso, dado que los valores ausentes som muy pocos y están muy dispersos, se emplea como estrategia de imputacion el *forward fill*. Existen otras estrategias que podrían dar mejores resultados.

In [11]:
datos = datos.fillna(method='ffill')

Una vez realizada la imputación de valores ausentes, se recalculan las variables mes, hora y día de la semana.

In [12]:
datos['month'] = datos.index.month 
datos['hour'] = datos.index.hour
datos['weekday'] = datos.index.weekday # Lunes=0, Domingo=6

In [13]:
datos

,holiday,workingday,weather,temp,atemp,hum,windspeed,users,month,hour,weekday
date_time,,,,,,,,,,,
2011-01-01 00:00:00,0.0,0.0,clear,9.84,14.395,81.0,0.0000,16.0,1,0,5
2011-01-01 01:00:00,0.0,0.0,clear,9.02,13.635,80.0,0.0000,40.0,1,1,5
2011-01-01 02:00:00,0.0,0.0,clear,9.02,13.635,80.0,0.0000,32.0,1,2,5
2011-01-01 03:00:00,0.0,0.0,clear,9.84,14.395,75.0,0.0000,13.0,1,3,5
2011-01-01 04:00:00,0.0,0.0,clear,9.84,14.395,75.0,0.0000,1.0,1,4,5
...,...,...,...,...,...,...,...,...,...,...,...
2012-12-31 19:00:00,0.0,1.0,mist,10.66,12.880,60.0,11.0014,119.0,12,19,0
2012-12-31 20:00:00,0.0,1.0,mist,10.66,12.880,60.0,11.0014,89.0,12,20,0
2012-12-31 21:00:00,0.0,1.0,clear,10.66,12.880,60.0,11.0014,90.0,12,21,0


In [14]:
datos.to_csv("bike_sharing_dataset_clean.csv")